In [ ]:
import numpy as np
import pandas as pd
from typing import List, Tuple, Optional, Dict
import os
import matplotlib.pyplot as plt
import japanize_matplotlib
from scipy import signal, integrate
import cv2
from scipy.stats import entropy
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.interpolate import UnivariateSpline

from skimage.metrics import structural_similarity as ssim
from skimage.color import rgb2lab
from scipy.spatial import ConvexHull

import pyVHR as vhr
from pyVHR.extraction.sig_processing import SignalProcessing
from pyVHR.plot.visualize import *
from pyVHR.BVP import *
vhr.plot.VisualizeParams.renderer = 'notebook'

import mediapipe as mp

In [ ]:
# 入力とする動画と動画のファイル名を取得
root_dir = "experimentData\\"
data_dirs = [os.path.join(root_dir, d) for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
movie_paths = []
movie_names = []
true_value_csv_array = []
true_value_rri_csv_array = []
print("動画ディレクトリ:", data_dirs)

for i in range(len(data_dirs)):
    data_dir = data_dirs[i]

    # 動画ファイルのパスを取得
    movie_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.avi')]
    movie_paths.extend(movie_files)

    movie_name = os.path.basename(data_dir)
    movie_names.append(movie_name)

    # ppgファイルのパスを取得
    movie_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.csv')]
    true_value_csv_array = [f.replace('.avi', '.csv') for f in movie_paths if f.endswith('.avi')]
    true_value_rri_csv_array.append(os.path.join(data_dir, 'RRI_Simple_' + movie_name + '.csv'))


f_1_ffi = 0.0399  # LFのはじめ
f_2 = 0.151  # LFの終わり、HFのはじめ
f_3 = 0.401  # HFの終わり

# start_index = 14
# # start_index = 17
# # end_index = 18
# end_index = len(movie_paths)
# # start_index = end_index - 1  # 最後の動画のみを対象とする

# data_dirs = data_dirs[start_index:end_index]
# movie_paths = movie_paths[start_index:end_index]
# movie_names = movie_names[start_index:end_index]
# true_value_csv_array = true_value_csv_array[start_index:end_index]
# true_value_rri_csv_array = true_value_rri_csv_array[start_index:end_index]

print(f"data_dirs: {data_dirs}")
print(f"movie_paths: {movie_paths}")
print(f"movie_names: {movie_names}")
print(f"true_value_csv_array: {true_value_csv_array}")
print(f"true_value_rri_csv_array: {true_value_rri_csv_array}")

In [ ]:
SAVE_DIR  = "rppgAccuracyEvalu"
WINDOW_SIZES = [2, 4, 6, 8, 10]
STRIDES = [2]

## 肌領域検出でRGB抽出

In [ ]:
def visualize_roi(frame, roi_info):
    """
    ROIを可視化する関数
    
    Parameters:
    -----------
    frame : np.ndarray
        元の画像フレーム
    roi_info : dict
        extract_rgb_with_fixed_roi()が返すROI情報
    
    Returns:
    --------
    vis_frame : np.ndarray
        ROIを描画した画像
    """
    vis_frame = frame.copy()
    
    # マスク領域を半透明で表示
    mask_colored = cv2.cvtColor(roi_info['mask'], cv2.COLOR_GRAY2BGR)
    mask_colored[:, :, 1] = roi_info['mask']  # 緑チャンネル
    vis_frame = cv2.addWeighted(vis_frame, 0.7, mask_colored, 0.3, 0)
    
    # 凸包の輪郭を描画
    cv2.polylines(vis_frame, [roi_info['hull_points']], 
                  True, (0, 255, 0), 2)
    
    # ランドマークを描画（オプション）
    for pt in roi_info['landmarks']:
        cv2.circle(vis_frame, tuple(pt), 1, (255, 0, 0), -1)
    
    return vis_frame

In [ ]:
# メイン処理
for i in range(len(movie_paths)):
    inputMoviePath = movie_paths[i]
    rootDir = data_dirs[i]
    dataName = movie_names[i]
    save_dir = os.path.join(rootDir, SAVE_DIR)
    print(f"\n{'='*60}")
    print(f"処理中: {dataName}")
    print(f"保存先: {save_dir}")
    print(f"{'='*60}")
    os.makedirs(save_dir, exist_ok=True)

    # 動画情報取得
    cap = cv2.VideoCapture(inputMoviePath)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps
    print(f"動画情報: {total_frames}フレーム, {fps:.2f}fps, {duration:.2f}秒")

    # 1フレーム目でROI検出
    ret, first_frame = cap.read()
    if not ret:
        cap.release()
        print(f"エラー: 動画の読み込みに失敗しました: {inputMoviePath}")
        continue
    
    mp_face_mesh = mp.solutions.face_mesh
    with mp_face_mesh.FaceMesh(
        static_image_mode=True,
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5) as face_mesh:
        
        frame_rgb = cv2.cvtColor(first_frame, cv2.COLOR_BGR2RGB)
        results_mp = face_mesh.process(frame_rgb)
        
        if not results_mp.multi_face_landmarks:
            cap.release()
            print(f"エラー: 顔が検出できませんでした: {inputMoviePath}")
            continue
        
        landmarks = results_mp.multi_face_landmarks[0]
        h, w = first_frame.shape[:2]
        
        ldmks = np.array([[int(l.x * w), int(l.y * h)] 
                        for l in landmarks.landmark])
        
        hull = ConvexHull(ldmks)
        hull_points = ldmks[hull.vertices]
        
        mask = np.zeros((h, w), dtype=np.uint8)
        cv2.fillConvexPoly(mask, hull_points, 255)

        roi_info = {
            'mask': mask,
            'hull_points': hull_points,
            'landmarks': ldmks
        }

        vis_frame = visualize_roi(first_frame, roi_info)
        roi_vis_path = os.path.join(save_dir, "roi_visualization.jpg")
        cv2.imwrite(roi_vis_path, vis_frame)
        print(f"ROI可視化画像を保存: {roi_vis_path}")
        print(f"1フレーム目でROI検出完了: {len(hull_points)}個の凸包頂点")

    # 全フレームでRGB信号抽出
    print("全フレームで特徴量抽出中...")
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    frame_count = 0

    # 結果を格納する辞書
    results = {
        'frame_number': [], 'timestamp': [],'r_mean': [], 'g_mean': [], 'b_mean': []
    }

    prev_frame = None
    prev_gray = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        timestamp = frame_count / fps
        
        # BGRをRGBに変換
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # マスク領域のピクセルを抽出
        valid_pixels = frame_rgb[mask > 0]
        
        if len(valid_pixels) > 0:
            # RGB統計量
            r_mean = np.mean(valid_pixels[:, 0])
            g_mean = np.mean(valid_pixels[:, 1])
            b_mean = np.mean(valid_pixels[:, 2])
            
        else:
            raise ValueError("ROI内に有効なピクセルが存在しません")

        # 結果を保存
        results['frame_number'].append(frame_count)
        results['timestamp'].append(timestamp)
        results['r_mean'].append(r_mean)
        results['g_mean'].append(g_mean)
        results['b_mean'].append(b_mean)
        
        # 前フレームを保存
        prev_frame = frame.copy()
        prev_gray = gray.copy()
        
        frame_count += 1
        
        if frame_count % 100 == 0:
            print(f"  処理中: {frame_count}/{total_frames} フレーム ({frame_count/total_frames*100:.1f}%)")

    cap.release()
    print(f"特徴量抽出完了: {frame_count}フレーム処理")

    # DataFrameに変換して保存
    signals_df = pd.DataFrame(results)

    save_path = os.path.join(save_dir, "extracted_signals.csv")
    signals_df.to_csv(save_path, index=False)
    print(f"抽出した信号を保存: {save_path}")

成果物
- 1フレームずつのRGBシグナル

## windowごとにRGB信号を分類

In [ ]:
def array_to_full_string(arr):
    """NumPy配列を省略なしの文字列に変換"""
    if isinstance(arr, str):
        return arr
    elif isinstance(arr, (np.ndarray, list)):
        # NumPy配列またはリストを完全な文字列に変換
        arr_np = np.array(arr)
        return np.array2string(arr_np, threshold=np.inf, max_line_width=np.inf, separator=' ')
    else:
        return str(arr)

In [ ]:
class StrideSegmentCalculator:
    def __init__(
        self,
        window_sizes: List[float] = [2, 3, 4, 5],
        strides: List[float] = [0.1, 0.5, 1, 1.5, 2],
    ):
        """
        Parameters:
        -----------
        window_sizes : List[float]
            窓幅（秒）のリスト
        strides : List[float]
            移動秒数のリスト
        """
        self.window_sizes = window_sizes
        self.strides = strides

    def calculate_overlap(self, window_size: float, stride: float) -> float:
        """
        窓幅と移動秒数からオーバーラップ率を計算

        Parameters:
        -----------
        window_size : float
            窓幅（秒）
        stride : float
            移動秒数（秒）

        Returns:
        --------
        float
            オーバーラップ率（%）
        """
        if stride >= window_size:
            return 0
        overlap = (window_size - stride) / window_size * 100
        return round(overlap, 2)

    def calculate_segments(
        self, window_size: float, stride: float, total_frames: int, fps: int
    ) -> List[Tuple[int, int]]:
        """
        フレーム数から解析区間を計算

        Parameters:
        -----------
        window_size : float
            窓幅（秒）
        stride : float
            移動秒数（秒）
        total_frames : int
            総フレーム数
        fps : int
            フレームレート

        Returns:
        --------
        List[Tuple[int, int]]
            各区間の(開始フレーム, 終了フレーム)のリスト
        """
        frames_per_window = round(window_size * fps)
        frames_per_stride = round(stride * fps)

        segments = []
        start_frame = 0

        while start_frame + frames_per_window <= total_frames:
            segments.append((start_frame, start_frame + frames_per_window))
            start_frame += frames_per_stride

        return segments

    def create_analysis_dataframe(self, total_frames: int, fps: int) -> pd.DataFrame:
        """
        全ての窓幅と移動秒数の組み合わせに対してDataFrameを生成

        Parameters:
        -----------
        total_frames : int
            総フレーム数
        fps : int
            フレームレート

        Returns:
        --------
        pd.DataFrame
            各条件でのセグメント情報を含むDataFrame
            columns: window_size, stride, overlap, segment_number, frame_start, frame_end
        """
        data_dict = {
            "window_size": [],
            "stride": [],
            "overlap": [],
            "segment_number": [],
            "frame_start": [],
            "frame_end": [],
        }

        for window_size in self.window_sizes:
            for stride in self.strides:
                overlap = self.calculate_overlap(window_size, stride)
                segments = self.calculate_segments(
                    window_size, stride, total_frames, fps
                )

                for i, (start_frame, end_frame) in enumerate(segments):
                    data_dict["window_size"].append(window_size)
                    data_dict["stride"].append(stride)
                    data_dict["overlap"].append(overlap)
                    data_dict["segment_number"].append(i)
                    data_dict["frame_start"].append(start_frame)
                    data_dict["frame_end"].append(end_frame)

        return pd.DataFrame(data_dict)


class PulseAnalysisDataStrides:
    def __init__(self, window_sizes, strides):
        # 窓枠とストライドの値を定義
        self.window_sizes = window_sizes
        self.strides = strides

        # accuracyのみを格納するDataFrameを初期化
        self.results = pd.DataFrame(
            index=pd.Index(self.window_sizes, name="window_size"),
            columns=pd.Index(self.strides, name="strides"),
        )

    def add_accuracy(self, window_size: float, strides: int, accuracy: float):
        """
        精度データを追加する

        Parameters:
        -----------
        window_size : float
            窓幅（秒）
        strides : int
            ストライド（s）
        accuracy : float
            精度値
        """
        self.results.loc[window_size, strides] = accuracy

    def _create_heatmap_dataframe(self):
        """
        ヒートマップ用のDataFrameを作成する内部メソッド

        Returns:
        --------
        pd.DataFrame
            ヒートマップ用に整形されたDataFrame
        """
        data = {"stride": self.strides}
        for window_size in self.window_sizes:
            data[window_size] = [
                self.results.loc[window_size, stride] for stride in self.strides
            ]
        df = pd.DataFrame(data).set_index("stride").T
        return df

    def save_heatmap(
        self,
        title: str,
        save_path: str,
        figsize: tuple = (10, 8),
        cmap: str = "YlGnBu",
        colorbar_label: str = "MAE",
    ):
        """
        ヒートマップを作成して保存する

        Parameters:
        -----------
        title : str
            プロットのタイトル
        save_path : str
            保存先のパス
        figsize : tuple, optional
            図のサイズ (default: (10, 8))
        cmap : str, optional
            カラーマップ (default: 'YlGnBu')
        colorbar_label : str, optional
            カラーバーのラベル (default: 'MAE')
        """
        df = self._create_heatmap_dataframe()

        # ヒートマップを作成
        plt.figure(figsize=figsize)
        sns.heatmap(
            df, annot=True, fmt=".4f", cmap=cmap, cbar_kws={"label": colorbar_label}
        )
        plt.title(f"{title}")
        plt.xlabel("Stride [s]")
        plt.ylabel("Window Size [s]")
        plt.tight_layout()
        plt.savefig(save_path, dpi=300)
        plt.close()

    def save_heatmap_std(self, title: str, save_path: str, figsize: tuple = (10, 8)):
        """
        標準偏差のヒートマップを作成して保存する

        Parameters:
        -----------
        title : str
            プロットのタイトル
        save_path : str
            保存先のパス
        figsize : tuple, optional
            図のサイズ (default: (10, 8))
        """
        self.save_heatmap(
            title,
            save_path,
            figsize,
            cmap="Reds",
            colorbar_label="Standard Deviation",
        )

In [ ]:
all_results_list = []
for i in range(len(movie_paths)):
    inputMoviePath = movie_paths[i]
    rootDir = data_dirs[i]
    dataName = movie_names[i]

    print(f'Processing movie: {inputMoviePath}')

    # 動画のfpsを取得
    cap = cv2.VideoCapture(inputMoviePath)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    samplingRate = fps
    cap.release()
    
    # CSVファイルの読み込み
    ecg_csv_path = os.path.join(rootDir, dataName + '.csv')
    ecg_df = pd.read_csv(ecg_csv_path)
    
    ecg_RRI_csv_path = os.path.join(rootDir, f'RRI_Simple_{movie_names[i]}.csv')
    ecg_RRI_df = pd.read_csv(ecg_RRI_csv_path)
    
    stride_segment_calculator = StrideSegmentCalculator(window_sizes=WINDOW_SIZES, strides=STRIDES)
    analysis_df = stride_segment_calculator.create_analysis_dataframe(total_frames, fps)

    # RGB信号の読み込み
    save_roi_dir = os.path.join(rootDir, SAVE_DIR)
    os.makedirs(save_roi_dir, exist_ok=True)
    signals_csv_path = os.path.join(save_roi_dir, f'extracted_signals.csv')
    signals_df = pd.read_csv(signals_csv_path)

    # 結果を格納するリスト
    all_window_results = []
    for idx, row in analysis_df.iterrows():
        window_size = row['window_size']
        frame_start = row['frame_start']
        frame_end = row['frame_end']
        stride = row['stride']

        # 窓の時間範囲を計算
        window_start_time = frame_start / fps
        window_end_time = frame_end / fps

        if(window_end_time > total_frames / fps):
            continue

        print(f"\nウィンドウ {idx}: 窓サイズ {window_size}s, ストライド {stride}s, フレーム {frame_start}-{frame_end}, 時間 {window_start_time:.2f}-{window_end_time:.2f}s")
            
        # 該当する窓の時間範囲内の真値RRIデータを抽出
        ecg_RRI_mask = (ecg_RRI_df['time'] >= window_start_time) & \
                (ecg_RRI_df['time'] < window_end_time)
        ecg_bpm_in_window = ecg_RRI_df[ecg_RRI_mask]['BPM'].values
        ecg_bpm_in_window_mean = np.mean(ecg_bpm_in_window) if len(ecg_bpm_in_window) > 0 else np.nan

        print(ecg_bpm_in_window)

        # 該当する窓の時間範囲内のRGB信号を抽出し、窓内でBVP算出
        bvp_mask = (signals_df['timestamp'] >= window_start_time) & (signals_df['timestamp'] < window_end_time)
        r_signal_in_window = signals_df[bvp_mask]['r_mean'].values
        g_signal_in_window = signals_df[bvp_mask]['g_mean'].values
        b_signal_in_window = signals_df[bvp_mask]['b_mean'].values


        # 窓情報を保存
        window_info = {
            'window_index': idx,
            'window_size': window_size,
            'stride': stride,
            'frame_start': frame_start,
            'frame_end': frame_end,
            'window_start_time': window_start_time,
            'window_end_time': window_end_time,
            'r_signal_in_window': array_to_full_string(r_signal_in_window),
            'g_signal_in_window': array_to_full_string(g_signal_in_window),
            'b_signal_in_window': array_to_full_string(b_signal_in_window),
            'ecg_bpm_in_window': array_to_full_string(ecg_bpm_in_window),
            'ecg_bpm_in_window_mean': ecg_bpm_in_window_mean,
        }
        all_window_results.append(window_info)

    # 全結果をDataFrameに変換して保存
    results_df = pd.DataFrame(all_window_results)
    results_csv_path = os.path.join(save_roi_dir, f'window_signals_{dataName}.csv')
    results_df.to_csv(results_csv_path, index=False, encoding='utf-8-sig')
    print(f"\n結果をCSVに保存: {results_csv_path}")

## 窓ごとにBVP解析

In [ ]:
def analyze_window_fft(values, fps):
    """
    時系列データの最大周波数とスペクトル情報を計算する関数

    Parameters:
    -----------
    values : array-like
        分析対象の時系列データ
    fps : int
        サンプリング周波数（1秒あたりのフレーム数）

    Returns:
    --------
    dict
        以下のキーを含む辞書：
        - 'max_freq': 検出された最大周波数
        - 'max_amplitude': 最大周波数のときの振幅
        - 'frequencies': 周波数配列（正の周波数のみ）
        - 'amplitudes': 振幅配列（正の周波数のみ）
        - 'power_spectrum': パワースペクトル
        - 'dominant_freqs': 上位5つの卓越周波数とその振幅
        - 'spectral_centroid': スペクトル重心
        - 'spectral_bandwidth': スペクトル帯域幅
        - 'total_power': 全体のパワー
    """
    # データをnumpy配列に変換
    values = np.array(values)

    # ゼロパディングで分解能を向上（窓長の8倍）
    n_pad = len(values) * 8

    # ハミング窓を適用（オプション：コメントアウトされている）
    # window = np.hamming(len(values))
    # windowed_data = values * window

    # FFTを実行（ゼロパディング適用）
    fft_result = np.fft.fft(values, n=n_pad)
    fft_freq = np.fft.fftfreq(n_pad, 1 / fps)

    # 正の周波数のみを取得
    positive_freq_idx = fft_freq > 0
    positive_fft = np.abs(fft_result[positive_freq_idx])
    positive_freq = fft_freq[positive_freq_idx]
    
    # パワースペクトルを計算
    power_spectrum = positive_fft ** 2

    # 最大周波数の検出と補間
    max_idx = np.argmax(positive_fft)
    max_amplitude = positive_fft[max_idx]
    
    if 0 < max_idx < len(positive_fft) - 1:
        # 3点を使用した放物線補間
        alpha = positive_fft[max_idx - 1]
        beta = positive_fft[max_idx]
        gamma = positive_fft[max_idx + 1]
        peak_pos = 0.5 * (alpha - gamma) / (alpha - 2 * beta + gamma)

        # 補間された周波数と振幅
        freq_resolution = fps / n_pad
        max_freq = positive_freq[max_idx] + peak_pos * freq_resolution
        
        # 補間された振幅（放物線の頂点）
        max_amplitude = beta - 0.25 * (alpha - gamma) * peak_pos
    else:
        max_freq = positive_freq[max_idx]
    
    # 上位5つの卓越周波数を検出
    top_indices = np.argsort(positive_fft)[-5:][::-1]
    dominant_freqs = [(positive_freq[idx], positive_fft[idx]) for idx in top_indices]
    
    # スペクトル特徴量の計算
    # スペクトル重心（周波数の重み付き平均）
    spectral_centroid = np.sum(positive_freq * positive_fft) / np.sum(positive_fft)
    
    # スペクトル帯域幅（重心からの重み付き分散）
    spectral_bandwidth = np.sqrt(
        np.sum(((positive_freq - spectral_centroid) ** 2) * positive_fft) / np.sum(positive_fft)
    )
    
    # 全体のパワー
    total_power = np.sum(power_spectrum)
    
    # 結果を辞書にまとめる
    result = {
        'max_freq': max_freq,
        'max_amplitude': max_amplitude,
        'frequencies': positive_freq,
        'amplitudes': positive_fft,
        'power_spectrum': power_spectrum,
        'dominant_freqs': dominant_freqs,
        'spectral_centroid': spectral_centroid,
        'spectral_bandwidth': spectral_bandwidth,
        'total_power': total_power,
        'freq_resolution': fps / n_pad,  # 周波数分解能
        'nyquist_freq': fps / 2  # ナイキスト周波数
    }
    
    return result

In [ ]:
def extract_BVPsignal(r_signals, g_signals, b_signals,  fps, deviceType, bvpMethod, bvpMethodName, method_params=None):
    """
    RGB信号からBVP信号を抽出する関数
    """
    rgb_signal = np.array([[r_signals, g_signals, b_signals]], dtype=np.float32)
    print(f"\nRGB信号の形状: {rgb_signal.shape}")
    
    signal_length = rgb_signal.shape[2]
    min_required_length = 50
    
    if signal_length < min_required_length:
        print(f"警告: 信号長が短すぎます ({signal_length} < {min_required_length})。処理をスキップします。")
        return None, None
    
    filtered_signal = [rgb_signal]
    
    # デフォルトのメソッドパラメータ
    if method_params is None:
        method_params = {}
    
    # メソッド別パラメータ設定
    if bvpMethodName in ["cupy_POS", "cpu_POS"]:
        method_params['fps'] = fps
    elif bvpMethodName in ["cpu_ICA", "cpu_PCA"]:
        method_params['component'] = 'all_comp'
    
    print(f"\nBVP抽出開始 (メソッド: {bvpMethodName})")
    print(f"パラメータ: {method_params}")
    
    # BVP信号抽出
    if method_params:
        bvp_signal = vhr.BVP.RGB_sig_to_BVP(
            filtered_signal,
            fps,
            device_type=deviceType,
            method=bvpMethod,
            params=method_params
        )
    else:
        bvp_signal = vhr.BVP.RGB_sig_to_BVP(
            filtered_signal,
            fps,
            device_type=deviceType,
            method=bvpMethod
        )
    
    # 生のBVP信号を保存
    raw_bvp_signal = bvp_signal[0].copy() if len(bvp_signal) > 0 else None
    
    # 後処理フィルタリング
    bvp_signal = vhr.BVP.apply_filter(
        bvp_signal,
        vhr.BVP.BPfilter,
        params={'order': 6, 'minHz': 0.5, 'maxHz': 2.0, 'fps': fps}
    )
    
    bvp_signal = vhr.BVP.apply_filter(bvp_signal, vhr.BVP.zeromean)
    
    filtered_bvp_signal = bvp_signal[0] if len(bvp_signal) > 0 else None
    
    print(f"\nBVP信号抽出完了")
    return raw_bvp_signal, filtered_bvp_signal

In [ ]:
# メソッドの組み合わせを定義
methodCombinations = [
    # 拡張POS（蒸気対応）
    ['cuda', cupy_CHROM, "cupy_CHROM"],
    ['cpu', cpu_LGI, "cpu_LGI"],
    ['cpu', cpu_GREEN, "cpu_GREEN"],
    ['cpu', cpu_ICA, "cpu_ICA"],
    ['cuda', cupy_POS, "cupy_POS"]
]

for i in range(len(movie_paths)):
    print(f'Processing movie: {movie_paths[i]}')
    inputMoviePath = movie_paths[i]
    rootDir = data_dirs[i]
    dataName = movie_names[i]

    cap = cv2.VideoCapture(inputMoviePath)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # CSVファイルの読み込み
    ecg_csv_path = os.path.join(rootDir, dataName + '.csv')
    ecg_df = pd.read_csv(ecg_csv_path)

    ecg_RRI_csv_path = os.path.join(rootDir, f'RRI_Simple_{dataName}.csv')
    ecg_RRI_df = pd.read_csv(ecg_RRI_csv_path)

    ecg_bpm_in_window = ecg_RRI_df['BPM']
    ecg_bpm_in_window_mean = ecg_RRI_df['BPM'].values.mean()

    # window_signalsの読み込み
    os.makedirs(os.path.join(rootDir, SAVE_DIR), exist_ok=True)
    window_signals_data_path = os.path.join(rootDir, SAVE_DIR, f'{"window_signals_" + dataName}.csv')
    window_signals_data_df = pd.read_csv(window_signals_data_path)
    print(f'Loaded window signals data from {window_signals_data_path}, {len(window_signals_data_df)} rows')

    stride_segment_calculator = StrideSegmentCalculator(window_sizes=WINDOW_SIZES, strides=STRIDES)
    analysis_df = stride_segment_calculator.create_analysis_dataframe(total_frames, fps)

    # 結果を格納するリスト
    all_window_results = []
    
    for idx, row in analysis_df.iterrows():
        window_size = row['window_size']
        frame_start = row['frame_start']
        frame_end = row['frame_end']
        stride = row['stride']

        window_start_time = frame_start / fps
        window_end_time = frame_end / fps

        if window_end_time > total_frames / fps:
            continue

        print(f"\nウィンドウ {idx}: 窓サイズ {window_size}s, ストライド {stride}s, フレーム {frame_start}-{frame_end}, 時間 {window_start_time:.2f}-{window_end_time:.2f}s")

        # 真値RRIデータを抽出
        ecg_RRI_mask = (ecg_RRI_df['time'] >= window_start_time) & \
                       (ecg_RRI_df['time'] < window_end_time)
        ecg_bpm_in_window = ecg_RRI_df[ecg_RRI_mask]['BPM'].values
        ecg_bpm_in_window_mean = np.mean(ecg_bpm_in_window) if len(ecg_bpm_in_window) > 0 else np.nan
        print(f'    True Value RRI count in window: {len(ecg_bpm_in_window)}, Mean BPM: {ecg_bpm_in_window_mean:.2f}')

        # RGB信号を抽出
        bvp_mask = window_signals_data_df['window_index'] == idx
        print(f'    BVP mask has {bvp_mask.sum()} matching rows')

        filtered_rows = window_signals_data_df[bvp_mask]

        r_signal_in_window = filtered_rows['r_signal_in_window'].values[0]
        g_signal_in_window = filtered_rows['g_signal_in_window'].values[0]
        b_signal_in_window = filtered_rows['b_signal_in_window'].values[0]

        # 文字列をNumPy配列に変換
        r_signal_in_window = np.fromstring(r_signal_in_window[1:-1], sep=' ')
        g_signal_in_window = np.fromstring(g_signal_in_window[1:-1], sep=' ')
        b_signal_in_window = np.fromstring(b_signal_in_window[1:-1], sep=' ')
        
        if len(r_signal_in_window) == 0 or len(g_signal_in_window) == 0 or len(b_signal_in_window) == 0:
            raise ValueError("RGB信号が窓内に存在しません")

        # BVPメソッドの設定
        for methodCombination in methodCombinations:
            deviceType = methodCombination[0] 
            bvpMethod = methodCombination[1] 
            bvpMethodName = methodCombination[2]
            method_params = methodCombination[3].copy() if len(methodCombination) > 3 else {}


            print(f"\n{'='*60}")
            print(f"実行メソッド: {bvpMethodName}")
            print(f"{'='*60}")

            # rgbからBVPを計算
            raw_bvp_signal_in_window, filtered_bvp_signal_in_window = extract_BVPsignal(
                r_signal_in_window,
                g_signal_in_window,
                b_signal_in_window,
                fps,
                deviceType,
                bvpMethod,
                bvpMethodName,
                method_params=method_params
            )
            
            # BVP信号の抽出に失敗した場合はスキップ
            if filtered_bvp_signal_in_window is None:
                print(f"ウィンドウ {idx} をスキップ: BVP信号の抽出に失敗")
                continue
            
            # FFT解析
            raw_bvp_signal_in_window = raw_bvp_signal_in_window.flatten() if raw_bvp_signal_in_window is not None else None
            filtered_bvp_signal_in_window = filtered_bvp_signal_in_window.flatten()
            fft_result_dic = analyze_window_fft(filtered_bvp_signal_in_window, fps)

            # MAEの計算
            rppg_bpm = fft_result_dic['max_freq'] * 60
            rppg_freq = fft_result_dic['frequencies']
            rppg_amplitude = fft_result_dic['amplitudes']
            rppg_pwd = fft_result_dic['power_spectrum']

            bpm_MAE = np.abs(ecg_bpm_in_window_mean - rppg_bpm) if not np.isnan(ecg_bpm_in_window_mean) else np.nan

            print(f"\n結果サマリー:")
            print(f"  ECG BPM: {ecg_bpm_in_window_mean:.2f}")
            print(f"  rPPG BPM: {rppg_bpm:.2f}")
            print(f"  MAE: {bpm_MAE:.2f}")

            # 窓情報を保存
            window_info = {
                'window_index': idx,
                'bvp_method': bvpMethodName,
                'window_size': window_size,
                'stride': stride,
                'frame_start': frame_start,
                'frame_end': frame_end,
                'window_start_time': window_start_time,
                'window_end_time': window_end_time,
                'r_signal_in_window': array_to_full_string(r_signal_in_window),
                'g_signal_in_window': array_to_full_string(g_signal_in_window),
                'b_signal_in_window': array_to_full_string(b_signal_in_window),
                'raw_bvp_in_window': raw_bvp_signal_in_window,
                'filtered_bvp_in_window': filtered_bvp_signal_in_window,
                'ecg_bpm_in_window': ecg_bpm_in_window,
                'ecg_bpm_mean': ecg_bpm_in_window_mean,
                'rppg_bpm': rppg_bpm,
                'bpm_MAE': bpm_MAE,
                'max_freq': fft_result_dic['max_freq'],
                'max_amplitude': fft_result_dic['max_amplitude'],
                'spectral_centroid': fft_result_dic['spectral_centroid'],
                'spectral_bandwidth': fft_result_dic['spectral_bandwidth'],
                'total_power': fft_result_dic['total_power']
            }
            all_window_results.append(window_info)
    
    # 全結果をDataFrameに変換して保存
    results_df = pd.DataFrame(all_window_results)
    results_save_dir = os.path.join(rootDir, SAVE_DIR)
    os.makedirs(results_save_dir, exist_ok=True)
    results_csv_path = os.path.join(results_save_dir, f'window_analysis_{dataName}.csv')
    results_df.to_csv(results_csv_path, index=False, encoding='utf-8-sig')
    print(f"\nFFT結果をCSVに保存: {results_csv_path}")  

    # 分割時間の設定(秒)
    SPLIT_TIME = 30

    # 全ての動画データを処理
    for i in range(len(movie_paths)):
        inputMoviePath = movie_paths[i]
        rootDir = data_dirs[i]
        dataName = movie_names[i]

        print(f'\n{"="*60}')
        print(f'Processing movie: {dataName}')
        print(f'{"="*60}')
        
        # 前半と後半でデータを分割
        results_df = pd.read_csv(os.path.join(rootDir, SAVE_DIR, f'window_analysis_{dataName}.csv'))
        first_half_mask = results_df['window_end_time'] <= SPLIT_TIME
        second_half_mask = results_df['window_start_time'] >= SPLIT_TIME

        first_half_df = results_df[first_half_mask].copy()
        second_half_df = results_df[second_half_mask].copy()

        print(f'前半データ数: {len(first_half_df)}, 後半データ数: {len(second_half_df)}')
        
        # ヒートマップを作成する関数
        def create_mae_heatmap(df, title_suffix, save_suffix):
            """
            BPM MAEのヒートマップを作成
            
            Parameters:
            -----------
            df : pd.DataFrame
                window_analysisデータ
            title_suffix : str
                タイトルに追加する文字列
            save_suffix : str
                保存ファイル名に追加する文字列
            """
            # 手法のリストを取得
            methods = df['bvp_method'].unique()
            window_sizes = sorted(df['window_size'].unique())
            
            # ヒートマップ用のデータを準備
            heatmap_data = pd.DataFrame(index=window_sizes, columns=methods)
            
            for window_size in window_sizes:
                for method in methods:
                    mask = (df['window_size'] == window_size) & (df['bvp_method'] == method)
                    mae_values = df[mask]['bpm_MAE'].dropna()
                    
                    if len(mae_values) > 0:
                        # 平均MAEを計算
                        mean_mae = mae_values.mean()
                        heatmap_data.loc[window_size, method] = mean_mae
            
            # データ型を数値に変換
            heatmap_data = heatmap_data.astype(float)
            
            # ヒートマップを作成
            plt.figure(figsize=(12, 8))
            sns.heatmap(
                heatmap_data,
                annot=True,
                fmt='.4f',
                cmap='YlOrRd',
                cbar_kws={'label': 'Mean MAE (BPM)'},
                vmin=0,
                vmax=25
            )
            
            plt.title(f'MAE vs BVP Method - {dataName} ({title_suffix})', fontsize=14, pad=20)
            plt.xlabel('BVP Method', fontsize=12)
            plt.ylabel('Window Size [s]', fontsize=12)
            plt.tight_layout()
            
            # 保存
            save_dir = os.path.join(rootDir, SAVE_DIR)
            save_path = os.path.join(save_dir, f'heatmap_mae_{save_suffix}_{dataName}.png')
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            plt.close()
            
            print(f'  ヒートマップを保存: {save_path}')
            
            return heatmap_data
        
        # 前半のヒートマップを作成
        if len(first_half_df) > 0:
            print('\n前半(ノイズなし)のヒートマップを作成中...')
            first_half_heatmap = create_mae_heatmap(
                first_half_df,
                'First Half (No Noise)',
                'first_half'
            )
        else:
            print('前半のデータが存在しません')
        
        # 後半のヒートマップを作成
        if len(second_half_df) > 0:
            print('\n後半(ノイズあり)のヒートマップを作成中...')
            second_half_heatmap = create_mae_heatmap(
                second_half_df,
                'Second Half (With Noise)',
                'second_half'
            )
        else:
            print('後半のデータが存在しません')
        
        # 統計情報を出力
        print(f'\n統計情報 - {dataName}')
        print('-' * 60)
        
        if len(first_half_df) > 0:
            print('前半(ノイズなし):')
            print(f'  平均MAE: {first_half_df["bpm_MAE"].mean():.4f}')
            print(f'  最小MAE: {first_half_df["bpm_MAE"].min():.4f}')
            print(f'  最大MAE: {first_half_df["bpm_MAE"].max():.4f}')
        
        if len(second_half_df) > 0:
            print('\n後半(ノイズあり):')
            print(f'  平均MAE: {second_half_df["bpm_MAE"].mean():.4f}')
            print(f'  最小MAE: {second_half_df["bpm_MAE"].min():.4f}')
            print(f'  最大MAE: {second_half_df["bpm_MAE"].max():.4f}')

    print('\n全ての処理が完了しました!')  

In [ ]:
# メソッドの組み合わせを定義
methodCombinations = [
    # 拡張POS(蒸気対応)
    ['cuda', cupy_CHROM, "cupy_CHROM"],
    ['cpu', cpu_LGI, "cpu_LGI"],
    ['cpu', cpu_GREEN, "cpu_GREEN"],
    ['cpu', cpu_ICA, "cpu_ICA"],
    ['cuda', cupy_POS, "cupy_POS"]
]

# 全データの結果を格納するリスト
all_data_results = []

for i in range(len(movie_paths)):
    print(f'Processing movie: {movie_paths[i]}')
    inputMoviePath = movie_paths[i]
    rootDir = data_dirs[i]
    dataName = movie_names[i]

    cap = cv2.VideoCapture(inputMoviePath)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # CSVファイルの読み込み
    ecg_csv_path = os.path.join(rootDir, dataName + '.csv')
    ecg_df = pd.read_csv(ecg_csv_path)

    ecg_RRI_csv_path = os.path.join(rootDir, f'RRI_Simple_{dataName}.csv')
    ecg_RRI_df = pd.read_csv(ecg_RRI_csv_path)

    ecg_bpm_in_window = ecg_RRI_df['BPM']
    ecg_bpm_in_window_mean = ecg_RRI_df['BPM'].values.mean()

    # window_signalsの読み込み
    os.makedirs(os.path.join(rootDir, SAVE_DIR), exist_ok=True)
    window_signals_data_path = os.path.join(rootDir, SAVE_DIR, f'{"window_signals_" + dataName}.csv')
    window_signals_data_df = pd.read_csv(window_signals_data_path)
    print(f'Loaded window signals data from {window_signals_data_path}, {len(window_signals_data_df)} rows')

    stride_segment_calculator = StrideSegmentCalculator(window_sizes=WINDOW_SIZES, strides=STRIDES)
    analysis_df = stride_segment_calculator.create_analysis_dataframe(total_frames, fps)

    # 結果を格納するリスト
    all_window_results = []
    
    for idx, row in analysis_df.iterrows():
        window_size = row['window_size']
        frame_start = row['frame_start']
        frame_end = row['frame_end']
        stride = row['stride']

        window_start_time = frame_start / fps
        window_end_time = frame_end / fps

        if window_end_time > total_frames / fps:
            continue

        print(f"\nウィンドウ {idx}: 窓サイズ {window_size}s, ストライド {stride}s, フレーム {frame_start}-{frame_end}, 時間 {window_start_time:.2f}-{window_end_time:.2f}s")

        # 真値RRIデータを抽出
        ecg_RRI_mask = (ecg_RRI_df['time'] >= window_start_time) & \
                       (ecg_RRI_df['time'] < window_end_time)
        ecg_bpm_in_window = ecg_RRI_df[ecg_RRI_mask]['BPM'].values
        ecg_bpm_in_window_mean = np.mean(ecg_bpm_in_window) if len(ecg_bpm_in_window) > 0 else np.nan
        print(f'    True Value RRI count in window: {len(ecg_bpm_in_window)}, Mean BPM: {ecg_bpm_in_window_mean:.2f}')

        # RGB信号を抽出
        bvp_mask = window_signals_data_df['window_index'] == idx
        print(f'    BVP mask has {bvp_mask.sum()} matching rows')

        filtered_rows = window_signals_data_df[bvp_mask]

        r_signal_in_window = filtered_rows['r_signal_in_window'].values[0]
        g_signal_in_window = filtered_rows['g_signal_in_window'].values[0]
        b_signal_in_window = filtered_rows['b_signal_in_window'].values[0]

        # 文字列をNumPy配列に変換
        r_signal_in_window = np.fromstring(r_signal_in_window[1:-1], sep=' ')
        g_signal_in_window = np.fromstring(g_signal_in_window[1:-1], sep=' ')
        b_signal_in_window = np.fromstring(b_signal_in_window[1:-1], sep=' ')
        
        if len(r_signal_in_window) == 0 or len(g_signal_in_window) == 0 or len(b_signal_in_window) == 0:
            raise ValueError("RGB信号が窓内に存在しません")

        # BVPメソッドの設定
        for methodCombination in methodCombinations:
            deviceType = methodCombination[0] 
            bvpMethod = methodCombination[1] 
            bvpMethodName = methodCombination[2]
            method_params = methodCombination[3].copy() if len(methodCombination) > 3 else {}

            print(f"\n{'='*60}")
            print(f"実行メソッド: {bvpMethodName}")
            print(f"{'='*60}")

            # rgbからBVPを計算
            raw_bvp_signal_in_window, filtered_bvp_signal_in_window = extract_BVPsignal(
                r_signal_in_window,
                g_signal_in_window,
                b_signal_in_window,
                fps,
                deviceType,
                bvpMethod,
                bvpMethodName,
                method_params=method_params
            )
            
            # BVP信号の抽出に失敗した場合はスキップ
            if filtered_bvp_signal_in_window is None:
                print(f"ウィンドウ {idx} をスキップ: BVP信号の抽出に失敗")
                continue
            
            # FFT解析
            raw_bvp_signal_in_window = raw_bvp_signal_in_window.flatten() if raw_bvp_signal_in_window is not None else None
            filtered_bvp_signal_in_window = filtered_bvp_signal_in_window.flatten()
            fft_result_dic = analyze_window_fft(filtered_bvp_signal_in_window, fps)

            # MAEの計算
            rppg_bpm = fft_result_dic['max_freq'] * 60
            rppg_freq = fft_result_dic['frequencies']
            rppg_amplitude = fft_result_dic['amplitudes']
            rppg_pwd = fft_result_dic['power_spectrum']

            bpm_MAE = np.abs(ecg_bpm_in_window_mean - rppg_bpm) if not np.isnan(ecg_bpm_in_window_mean) else np.nan

            print(f"\n結果サマリー:")
            print(f"  ECG BPM: {ecg_bpm_in_window_mean:.2f}")
            print(f"  rPPG BPM: {rppg_bpm:.2f}")
            print(f"  MAE: {bpm_MAE:.2f}")

            # 窓情報を保存
            window_info = {
                'data_name': dataName,  # データ名を追加
                'window_index': idx,
                'bvp_method': bvpMethodName,
                'window_size': window_size,
                'stride': stride,
                'frame_start': frame_start,
                'frame_end': frame_end,
                'window_start_time': window_start_time,
                'window_end_time': window_end_time,
                'r_signal_in_window': array_to_full_string(r_signal_in_window),
                'g_signal_in_window': array_to_full_string(g_signal_in_window),
                'b_signal_in_window': array_to_full_string(b_signal_in_window),
                'raw_bvp_in_window': raw_bvp_signal_in_window,
                'filtered_bvp_in_window': filtered_bvp_signal_in_window,
                'ecg_bpm_in_window': ecg_bpm_in_window,
                'ecg_bpm_mean': ecg_bpm_in_window_mean,
                'rppg_bpm': rppg_bpm,
                'bpm_MAE': bpm_MAE,
                'max_freq': fft_result_dic['max_freq'],
                'max_amplitude': fft_result_dic['max_amplitude'],
                'spectral_centroid': fft_result_dic['spectral_centroid'],
                'spectral_bandwidth': fft_result_dic['spectral_bandwidth'],
                'total_power': fft_result_dic['total_power']
            }
            all_window_results.append(window_info)
            all_data_results.append(window_info)  # 全データ用にも追加
    
    # 個別データの結果をDataFrameに変換して保存
    results_df = pd.DataFrame(all_window_results)
    results_save_dir = os.path.join(rootDir, SAVE_DIR)
    os.makedirs(results_save_dir, exist_ok=True)
    results_csv_path = os.path.join(results_save_dir, f'window_analysis_{dataName}.csv')
    results_df.to_csv(results_csv_path, index=False, encoding='utf-8-sig')
    print(f"\nFFT結果をCSVに保存: {results_csv_path}")

# 全データ統合の結果をDataFrameに変換
print(f'\n{"="*60}')
print('全データ統合処理を開始')
print(f'{"="*60}')

all_results_df = pd.DataFrame(all_data_results)

# 統合結果を保存(最初のデータのディレクトリに保存)
integrated_save_dir = os.path.join(root_dir, SAVE_DIR)
os.makedirs(integrated_save_dir, exist_ok=True)
integrated_csv_path = os.path.join(integrated_save_dir, 'window_analysis_integrated_all.csv')
all_results_df.to_csv(integrated_csv_path, index=False, encoding='utf-8-sig')
print(f'統合結果をCSVに保存: {integrated_csv_path}')
print(f'統合データ総数: {len(all_results_df)}')

# 分割時間の設定(秒)
SPLIT_TIME = 30

# 統合データを前半と後半に分割
first_half_mask = all_results_df['window_end_time'] <= SPLIT_TIME
second_half_mask = all_results_df['window_start_time'] >= SPLIT_TIME

integrated_first_half_df = all_results_df[first_half_mask].copy()
integrated_second_half_df = all_results_df[second_half_mask].copy()

print(f'統合前半データ数: {len(integrated_first_half_df)}, 統合後半データ数: {len(integrated_second_half_df)}')

# ヒートマップを作成する関数
def create_mae_heatmap(df, title_suffix, save_suffix, save_dir):
    """
    BPM MAEのヒートマップを作成
    
    Parameters:
    -----------
    df : pd.DataFrame
        window_analysisデータ
    title_suffix : str
        タイトルに追加する文字列
    save_suffix : str
        保存ファイル名に追加する文字列
    save_dir : str
        保存先ディレクトリ
    """
    # 手法のリストを取得
    methods = sorted(df['bvp_method'].unique())
    window_sizes = sorted(df['window_size'].unique())
    
    # ヒートマップ用のデータを準備
    heatmap_data = pd.DataFrame(index=window_sizes, columns=methods)
    count_data = pd.DataFrame(index=window_sizes, columns=methods)  # サンプル数も記録
    
    for window_size in window_sizes:
        for method in methods:
            mask = (df['window_size'] == window_size) & (df['bvp_method'] == method)
            mae_values = df[mask]['bpm_MAE'].dropna()
            
            if len(mae_values) > 0:
                # 平均MAEを計算
                mean_mae = mae_values.mean()
                heatmap_data.loc[window_size, method] = mean_mae
                count_data.loc[window_size, method] = len(mae_values)
    
    # データ型を数値に変換
    heatmap_data = heatmap_data.astype(float)
    
    # ヒートマップを作成
    plt.figure(figsize=(14, 8))
    sns.heatmap(
        heatmap_data,
        annot=True,
        fmt='.4f',
        cmap='YlOrRd',
        cbar_kws={'label': 'Mean MAE (BPM)'},
        vmin=0,
        vmax=25
    )
    
    plt.title(f'MAE vs BVP Method - Integrated All Data ({title_suffix})', fontsize=14, pad=20)
    plt.xlabel('BVP Method', fontsize=12)
    plt.ylabel('Window Size [s]', fontsize=12)
    plt.tight_layout()
    
    # 保存
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f'heatmap_mae_{save_suffix}_integrated_all.png')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f'  統合ヒートマップを保存: {save_path}')
    
    # サンプル数の情報も出力
    print(f'\n  各条件のサンプル数:')
    print(count_data.to_string())
    
    return heatmap_data

# 統合データの前半のヒートマップを作成
if len(integrated_first_half_df) > 0:
    print('\n統合データ - 前半(ノイズなし)のヒートマップを作成中...')
    integrated_first_half_heatmap = create_mae_heatmap(
        integrated_first_half_df,
        'First Half (No Noise) - All Data',
        'first_half',
        integrated_save_dir
    )
else:
    print('統合データの前半データが存在しません')

# 統合データの後半のヒートマップを作成
if len(integrated_second_half_df) > 0:
    print('\n統合データ - 後半(ノイズあり)のヒートマップを作成中...')
    integrated_second_half_heatmap = create_mae_heatmap(
        integrated_second_half_df,
        'Second Half (With Noise) - All Data',
        'second_half',
        integrated_save_dir
    )
else:
    print('統合データの後半データが存在しません')

# 統合データの統計情報を出力
print(f'\n統計情報 - 統合データ(全データ)')
print('-' * 60)

if len(integrated_first_half_df) > 0:
    print('前半(ノイズなし):')
    print(f'  データ数: {len(integrated_first_half_df)}')
    print(f'  平均MAE: {integrated_first_half_df["bpm_MAE"].mean():.4f}')
    print(f'  標準偏差: {integrated_first_half_df["bpm_MAE"].std():.4f}')
    print(f'  最小MAE: {integrated_first_half_df["bpm_MAE"].min():.4f}')
    print(f'  最大MAE: {integrated_first_half_df["bpm_MAE"].max():.4f}')
    print(f'  中央値: {integrated_first_half_df["bpm_MAE"].median():.4f}')

if len(integrated_second_half_df) > 0:
    print('\n後半(ノイズあり):')
    print(f'  データ数: {len(integrated_second_half_df)}')
    print(f'  平均MAE: {integrated_second_half_df["bpm_MAE"].mean():.4f}')
    print(f'  標準偏差: {integrated_second_half_df["bpm_MAE"].std():.4f}')
    print(f'  最小MAE: {integrated_second_half_df["bpm_MAE"].min():.4f}')
    print(f'  最大MAE: {integrated_second_half_df["bpm_MAE"].max():.4f}')
    print(f'  中央値: {integrated_second_half_df["bpm_MAE"].median():.4f}')

# 各データの内訳も表示
print('\n各データの内訳:')
for data_name in all_results_df['data_name'].unique():
    data_count = len(all_results_df[all_results_df['data_name'] == data_name])
    print(f'  {data_name}: {data_count}件')

print('\n全ての処理が完了しました!')